In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
train = pd.read_csv('F:\\JinnanPreliminary\\jinnan_round1_train_20181227.csv', encoding = 'gb18030')
#
test  = pd.read_csv('F:\\JinnanPreliminary\\jinnan_round1_testB_20190121.csv', encoding = 'gb18030')
sample_id=test["样本id"]
target_col = "收率"

# 删除异常值
print(train[train['收率'] < 0.87])

train = train[train['收率'] > 0.87]
train.loc[train['B14'] == 40, 'B14'] = 400
train = train[train['B14']>=400]

# 合并数据集, 顺便处理异常数据
target = train['收率']
train.loc[train['A25'] == '1900/3/10 0:00', 'A25'] = train['A25'].value_counts().values[0]
train['A25'] = train['A25'].astype(int)
train.loc[train['B14'] == 40, 'B14'] = 400
# test.loc[test['B14'] == 385, 'B14'] = 385

test_select = {}
for v in [280, 385, 390, 785]:
    print(v)
    print(test[test['B14'] == v]['样本id'])
    test_select[v] = test[test['B14'] == v]['样本id'].index
    print(test[test['B14'] == v]['样本id'].index)
    print(test_select[v])

del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

def timeTranSecond(t):
    try:
        t, m, s = t.split(":")
    except:
        if t == '1900/1/9 7:00':
            return 7 * 3600 / 3600
        elif t == '1900/1/1 2:30':
            return (2 * 3600 + 30 * 60) / 3600
        elif t == -1:
            return -1
        else:
            return 0

    try:
        tm = (int(t) * 3600 + int(m) * 60 + int(s)) / 3600
    except:
        return (30 * 60) / 3600

    return tm


for f in ['A5', 'A7', 'A9', 'A11', 'A14', 'A16', 'A24', 'A26', 'B5', 'B7']:
    try:
        data[f] = data[f].apply(timeTranSecond)
    except:
        print(f, '应该在前面被删除了！')


def getDuration(se):
    try:
        sh, sm, eh, em = re.findall(r"\d+\.?\d*", se)
    except:
        if se == -1:
            return -1

    try:
        if int(sh) > int(eh):
            tm = (int(eh) * 3600 + int(em) * 60 - int(sm) * 60 - int(sh) * 3600) / 3600 + 24
        else:
            tm = (int(eh) * 3600 + int(em) * 60 - int(sm) * 60 - int(sh) * 3600) / 3600
    except:
        if se == '19:-20:05':
            return 1
        elif se == '15:00-1600':
            return 1
    return tm


for f in ['A20', 'A28', 'B4', 'B9', 'B10', 'B11']:
    data[f] = data.apply(lambda df: getDuration(df[f]), axis=1)

data['样本id'] = data['样本id'].apply(lambda x: x.split('_')[1])
data['样本id'] = data['样本id'].astype(int)
data.drop(['样本id'],axis=1,inplace=True)
df_data=data.copy()

categorical_columns = [f for f in data.columns if f not in ['样本id']]
numerical_columns = [f for f in data.columns if f not in categorical_columns]
#label encoder
for f in categorical_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
# 基本数据处理完毕,

             样本id   A1     A2     A3   A4        A5    A6   A7  A8        A9  \
120    sample_842  300  125.0    NaN  980  14:30:00  24.0  NaN NaN  16:00:00   
138   sample_1001  300    NaN  405.0  700   1:00:00  21.0  NaN NaN   2:30:00   
248   sample_1040  200    NaN  270.0  470   9:30:00  21.0  NaN NaN  11:00:00   
322    sample_996  300    NaN  405.0  700   4:00:00  30.0  NaN NaN   5:00:00   
447    sample_994  300    NaN  405.0  700  10:40:00  28.0  NaN NaN  11:50:00   
484    sample_752  300    NaN  405.0  700  20:00:00  28.0  NaN NaN  22:00:00   
687    sample_748  300    NaN  405.0  700  15:00:00  30.0  NaN NaN  17:00:00   
762    sample_849  300  125.0    NaN  980   5:30:00  21.0  NaN NaN   7:00:00   
869    sample_293  300  125.0    NaN  980  15:00:00  21.0  NaN NaN  16:00:00   
956    sample_450  300    NaN  405.0  700  23:00:00  21.0  NaN NaN   0:30:00   
1012   sample_751  300    NaN  405.0  700   8:30:00  28.0  NaN NaN  10:30:00   
1195   sample_750  300    NaN  405.0  70

In [22]:
#add_minus
# numerical_col_list=['A6', 'A8', 'A10', 'A12', 'A15', 'A17', 'A19', 'A21', 'A22', 'A25', 'A27', 'B1', 'B6', 'B8', 'B12', 'B14']

# for col,index in zip(numerical_col_list,range(len(numerical_col_list))):
#     for _col in numerical_col_list[index:]:
#         data[col+"add"+_col]=df_data[col]+df_data[_col]
#         data[col+"minus"+_col]=df_data[col]-df_data[_col]
#         print(col,'+',_col)
#         print(col,'-',_col)
# print("-------------------------------------------",data.shape,"-------------------------------------------")

#unreasonable feature
data['b14/a1_a3_a4_a19_b1_b12'] = df_data['B14']/(df_data['A1']+df_data['A3']+df_data['A4']+df_data['A19']+df_data['B1']+df_data['B12'])
print("-------------------------------------------",data.shape,"-------------------------------------------")

------------------------------------------- (1479, 43) -------------------------------------------


In [3]:
# 特征提取一段结束 开始拼接数据
train = data[:train.shape[0]]
test  = data[train.shape[0]:]
train['target'] = list(target)
print(train.shape)
print(test.shape)

(1330, 43)
(149, 42)


E:\anaconda\install\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [24]:
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
train.columns

li = ['intTarget_0','intTarget_1','intTarget_2','intTarget_3','intTarget_4']
mean_columns = []
for f1 in categorical_columns:
    cate_rate = train[f1].value_counts(normalize=True, dropna=False).values[0]
    if cate_rate < 0.90:
        for f2 in li:
            col_name = 'B14_to_'+f1+"_"+f2+'_mean'
            mean_columns.append(col_name)
            order_label = train.groupby([f1])[f2].mean()
            print("order_label",order_label)
            train[col_name] = train['B14'].map(order_label)
            print("B14",train['B14'])
            print("train",col_name,train[col_name])
            miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
            if miss_rate > 0:
                train = train.drop([col_name], axis=1)
                mean_columns.remove(col_name)
            else:
                test[col_name] = test['B14'].map(order_label)
train.drop(li, axis=1, inplace=True)

E:\anaconda\install\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\anaconda\install\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


order_label A5
0     0.400000
1     0.054475
2     0.333333
3     0.102410
4     0.173913
5     0.111111
6     0.063197
7     0.375000
8     0.190476
9     0.151163
10    0.166667
11    0.400000
12    0.000000
13    0.500000
14    0.054054
15    0.028571
16    0.250000
17    0.210526
18    0.428571
19    0.428571
20    1.000000
21    0.166667
22    0.357143
23    0.272727
24    0.200000
25    0.333333
26    0.000000
27    0.250000
28    0.333333
29    0.333333
        ...   
33    0.666667
34    0.222222
35    0.333333
36    0.600000
37    0.238095
38    1.000000
39    0.181818
40    0.166667
41    0.500000
42    0.000000
43    0.090909
44    0.200000
45    0.000000
46    0.000000
47    0.000000
48    0.000000
49    0.285714
50    1.000000
51    1.000000
52    0.500000
53    1.000000
54    0.500000
55    0.000000
56    0.500000
57    1.000000
58    0.000000
59    0.000000
60    0.000000
61    0.500000
62    0.000000
Name: intTarget_0, Length: 63, dtype: float64
B14 0       0
1       0


order_label A5
0     0.000000
1     0.105058
2     0.000000
3     0.192771
4     0.000000
5     0.000000
6     0.026022
7     0.000000
8     0.000000
9     0.000000
10    0.083333
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.028571
16    0.000000
17    0.000000
18    0.047619
19    0.000000
20    0.000000
21    0.000000
22    0.000000
23    0.090909
24    0.000000
25    0.066667
26    0.000000
27    0.000000
28    0.000000
29    0.000000
        ...   
33    0.000000
34    0.000000
35    0.066667
36    0.000000
37    0.142857
38    0.000000
39    0.090909
40    0.000000
41    0.000000
42    0.000000
43    0.181818
44    0.000000
45    0.285714
46    0.000000
47    0.000000
48    0.000000
49    0.000000
50    0.000000
51    0.000000
52    0.000000
53    0.000000
54    0.000000
55    0.000000
56    0.000000
57    0.000000
58    0.000000
59    0.000000
60    0.000000
61    0.000000
62    0.000000
Name: intTarget_4, Length: 63, dtype: float64
B14 0       0
1       0


order_label A9
0     0.466667
1     0.040984
2     0.125000
3     0.093960
4     0.111111
5     0.086957
6     0.043651
7     0.280000
8     0.187500
9     0.166667
10    0.333333
11    0.300000
12    0.222222
13    0.111111
14    0.142857
15    0.125000
16    0.307692
17    0.250000
18    0.333333
19    1.000000
20    0.200000
21    0.333333
22    0.500000
23    0.100000
24    0.000000
25    0.176471
26    0.285714
27    0.400000
28    0.428571
29    0.230769
        ...   
58    0.363636
59    1.000000
60    0.000000
61    0.000000
62    0.000000
63    0.500000
64    0.500000
65    0.272727
66    0.000000
67    1.000000
68    0.500000
69    0.000000
70    1.000000
71    0.000000
72    0.000000
73    0.500000
74    1.000000
75    0.000000
76    0.000000
77    1.000000
78    0.000000
79    0.000000
80    1.000000
81    0.000000
82    1.000000
83    0.000000
84    0.000000
85    0.000000
86    0.000000
87    0.000000
Name: intTarget_0, Length: 88, dtype: float64
B14 0       0
1       0


order_label A9
0     0.066667
1     0.192623
2     0.250000
3     0.161074
4     0.333333
5     0.043478
6     0.067460
7     0.000000
8     0.000000
9     0.100000
10    0.000000
11    0.000000
12    0.000000
13    0.222222
14    0.000000
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.000000
21    0.000000
22    0.000000
23    0.200000
24    0.000000
25    0.117647
26    0.000000
27    0.000000
28    0.142857
29    0.153846
        ...   
58    0.000000
59    0.000000
60    0.000000
61    0.000000
62    0.000000
63    0.000000
64    0.000000
65    0.000000
66    0.000000
67    0.000000
68    0.000000
69    0.000000
70    0.000000
71    0.000000
72    0.000000
73    0.000000
74    0.000000
75    0.000000
76    0.000000
77    0.000000
78    0.000000
79    0.000000
80    0.000000
81    0.000000
82    0.000000
83    0.000000
84    0.000000
85    0.000000
86    0.000000
87    0.000000
Name: intTarget_3, Length: 88, dtype: float64
B14 0       0
1       0


...

order_label B8
0     0.440952
1     0.307692
2     0.083333
3     0.173228
4     0.400000
5     0.000000
6     0.142857
7     0.400000
8     0.100000
9     0.000000
10    0.266667
11    0.250000
12    0.166667
13    0.000000
14    0.250000
15    0.000000
16    0.142857
17    0.000000
18    0.000000
19    0.000000
20    0.000000
21    0.500000
22    1.000000
23    1.000000
24    0.500000
Name: intTarget_2, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
13

order_label B9
0     0.091753
1     0.266667
2     0.150538
3     0.019231
4     0.000000
5     0.173913
6     0.041667
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
Name: intTarget_3, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_to_B9_intTarget_3_mean 0       0.091753
1       0.091753
2      

order_label B10
0     0.093649
1     0.163717
2     0.200000
3     0.000000
4     0.014706
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.500000
14    0.000000
15    0.000000
16    0.000000
Name: intTarget_3, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_to_B10_intTarget_3_mean 0       0

order_label B12
0    0.450652
1    0.209845
2    0.500000
3    1.000000
Name: intTarget_1, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_to_B12_intTarget_1_mean 0       0.450652
1       0.450652
2       0.450652
3       0.450652
4       0.209845
5       0.209845
6       0.450652
7       0.450652
8       0.209845
9       0.450652
10      0.450652
11      0.450652
12  

order_label B14
0    0.004065
1    0.200608
2    0.159292
3    0.628571
4    0.000000
5    0.000000
Name: intTarget_3, dtype: float64
B14 0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      2
21      2
22      2
23      1
24      1
25      1
26      2
27      1
28      0
29      0
       ..
1300    2
1301    0
1302    0
1303    0
1304    1
1305    0
1306    0
1307    0
1308    2
1309    0
1310    0
1311    0
1312    0
1313    0
1314    0
1315    1
1316    0
1317    0
1318    1
1319    3
1320    1
1321    0
1322    2
1323    0
1324    3
1325    2
1326    1
1327    0
1328    0
1329    0
Name: B14, Length: 1330, dtype: int64
train B14_to_B14_intTarget_3_mean 0       0.004065
1       0.004065
2       0.004065
3       0.004065
4       0.200608
5       0.200608
6       0.004065
7       0.004065
8       0.200608
9       0.004065
10      0.

In [4]:
label_array=train['target']
train.drop(['target'],axis=1,inplace=True)
train_array=train.values
test_array=test.values
print(train.shape,test.shape)
print(train_array.shape,test_array.shape)

(1330, 42) (149, 42)
(1330, 42) (149, 42)


E:\anaconda\install\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
from sklearn.model_selection import KFold, RepeatedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [38]:
from sklearn.model_selection import GridSearchCV
def GridSearch(clf, params, X, y):
    cscv = GridSearchCV(clf, params, scoring='neg_mean_squared_error', n_jobs=1, cv=5)
    cscv.fit(X, y)

    print(cscv.cv_results_)
    print(cscv.best_params_)
    
from sklearn.model_selection import  RandomizedSearchCV
def RandomSearch(clf, params, X, y):
    rscv = RandomizedSearchCV(clf, params, scoring='neg_mean_squared_error', n_jobs=-1, cv=5,n_iter=10000)
    rscv.fit(X, y)

    print(rscv.cv_results_)
    print(rscv.best_params_)


In [39]:
xgb_params = {
              'eta': 0.005,
              'gamma ': 0,
              'max_depth': 10,
              'min_child_weight': 1,
              'subsample': 0.8,
              'colsample_bytree': 0.8,

              'objective': 'reg:linear',
              'eval_metric': 'rmse',
              
              "booster": "gbtree",
              'silent': True,
              'nthread': 4
            }
regr=xgb.XGBRegressor(**xgb_params)
adj_params = {
                'eta':np.arange(0.001,0.3,0.001,),
                'n_estimators':range(80,500,4),
                 'min_child_weight': range(3, 20, 2),
                 'colsample_bytree': np.arange(0.4, 1.0, 0.1),
                 'max_depth': range(3, 15, 1),
                 'subsample': np.arange(0.5, 1.0, 0.1),
             }
# GridSearch(regr , adj_params , train_array, label_array)
RandomSearch(regr , adj_params , train_array, label_array)


{'mean_fit_time': array([0.51099243, 0.48798971, 0.46659212, ..., 0.82238874, 0.72298942,
       0.36779308]), 'std_fit_time': array([0.21684475, 0.01730555, 0.01242017, ..., 0.11790671, 0.01328168,
       0.0768382 ]), 'mean_score_time': array([0.00360003, 0.00219979, 0.00180063, ..., 0.00239973, 0.00240307,
       0.00180063]), 'std_score_time': array([0.00135677, 0.00039892, 0.00040047, ..., 0.00049014, 0.00048802,
       0.00039988]), 'param_subsample': masked_array(data=[0.7999999999999999, 0.8999999999999999, 0.6, ..., 0.6,
                   0.6, 0.6],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[156, 252, 108, ..., 208, 356, 244],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[7, 13, 11, ..., 13, 17, 5],
             mask=[False, False, False, ..., False

In [9]:
# xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8,
#           'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}
xgb_params = {'colsample_bytree': 0.7601519554269048, 'eta': 0.05051192268121627, 'max_depth': 7, 'min_child_weight': 13.872262290113438, 'subsample': 0.7618029707600799,
    'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(train_array.shape[0])
predictions_xgb = np.zeros(test_array.shape[0])
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = xgb.DMatrix(train_array[trn_idx], label_array[trn_idx])
    val_data = xgb.DMatrix(train_array[val_idx], label_array[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200,
                    verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(train_array[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(test_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, label_array)))

fold n°1
[0]	train-rmse:0.404794	valid_data-rmse:0.405056
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.012225	valid_data-rmse:0.015166
[200]	train-rmse:0.010032	valid_data-rmse:0.014874
[300]	train-rmse:0.008993	valid_data-rmse:0.014874
[400]	train-rmse:0.00836	valid_data-rmse:0.014966
Stopping. Best iteration:
[243]	train-rmse:0.0095	valid_data-rmse:0.014791

fold n°2
[0]	train-rmse:0.405138	valid_data-rmse:0.403549
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.012189	valid_data-rmse:0.013499
[200]	train-rmse:0.010039	valid_data-rmse:0.013276
[300]	train-rmse:0.009086	valid_data-rmse:0.013392
Stopping. Best iteration:
[186]	train-rmse:0.010249	valid_data-rmse:0.013254

fold n°3
[0]	train-rmse:0.404687	valid_data-rmse:0.405378
Mult

In [10]:
#CV score: 0.00018679

# random 
#100
CV score: 0.00018763
CV score: 0.00019227
#1000
#     'subsample': 0.8999999999999999, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 7, 'eta': 0.081, 'colsample_bytree': 0.7,
CV score: 0.00018700
#     'subsample': 0.8999999999999999, 'n_estimators': 108, 'min_child_weight': 3, 'max_depth': 7, 'eta': 0.077, 'colsample_bytree': 0.4,
CV score: 0.00018343
#     'subsample': 0.8999999999999999, 'n_estimators': 184, 'min_child_weight': 3, 'max_depth': 5, 'eta': 0.136, 'colsample_bytree': 0.6,
CV score: 0.00018987   
#10000
#    'subsample': 0.8999999999999999, 'n_estimators': 172, 'min_child_weight': 3, 'max_depth': 5, 'eta': 0.097, 'colsample_bytree': 0.4,
CV score: 0.00018576
    
#bayes
CV score: 0.00019080
# 'colsample_bytree': 0.7601519554269048, 'eta': 0.05051192268121627, 'max_depth': 7, 'min_child_weight': 13.872262290113438, 'subsample': 0.7618029707600799,


In [11]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
def BayesianSearch(clf, params):
#   """贝叶斯优化器"""
    # 迭代次数
    num_iter = 25
    init_points = 5
    # 创建一个贝叶斯优化对象，输入为自定义的模型评估函数与超参数的范围
    bayes = BayesianOptimization(clf, params)
    # 开始优化
    bayes.maximize(init_points=init_points, n_iter=num_iter)
    # 输出结果
    params = bayes.res['max']
    print(params['max_params'])
    
    return params    

def GBM_evaluate(eta, min_child_weight, colsample_bytree, max_depth, subsample,):
   # """自定义的模型评估函数"""
    
    # 模型固定的超参数
    param = {
         'objective': 'reg:linear',
         'eval_metric': 'rmse',
         "booster": "gbtree",
         'silent': True,
         'nthread': 4
        }

    # 贝叶斯优化器生成的超参数
    param['min_child_weight'] = int(min_child_weight)
    param['colsample_bytree'] = float(colsample_bytree),
    param['max_depth'] = int(max_depth),
    param['subsample'] = float(subsample),
    param['eta'] = float(eta)

    # 5-flod 交叉检验，注意BayesianOptimization会向最大评估值的方向优化，因此对于回归任务需要取负数。
    # 这里的评估函数为neg_mean_squared_error，即负的MSE。
    val = cross_val_score(xgb.XGBRegressor(**xgb_params),
        train_array, label_array ,scoring='neg_mean_squared_error', cv=5).mean()

    return val

adj_params = {
             'eta': (0.001,0.3),
             'min_child_weight':(3, 20),
             'colsample_bytree':(0.4, 1.0),
             'max_depth': (3, 15),
             'subsample': (0.5, 1.0),
             }
# BayesianSearch(GBM_evaluate, adj_params)
num_iter = 100
init_points = 10
# 创建一个贝叶斯优化对象，输入为自定义的模型评估函数与超参数的范围
bayes = BayesianOptimization(GBM_evaluate, adj_params)
# 开始优化
bayes.maximize(init_points=init_points, n_iter=num_iter)
# 输出结果
# params = bayes.res['max']
params = bayes.max
print(params)
  

|   iter    |  target   | colsam... |    eta    | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
|  1        | -0.000188 |  0.4765   |  0.1319   |  5.599    |  9.343    |  0.6443   |
|  2        | -0.000188 |  0.5768   |  0.149    |  5.787    |  9.636    |  0.9844   |
|  3        | -0.000188 |  0.6088   |  0.1063   |  5.522    |  10.33    |  0.6965   |
|  4        | -0.000188 |  0.9468   |  0.07648  |  5.206    |  9.27     |  0.566    |
|  5        | -0.000188 |  0.7618   |  0.09822  |  6.524    |  14.9     |  0.6873   |
|  6        | -0.000188 |  0.5819   |  0.09243  |  8.024    |  9.405    |  0.9945   |
|  7        | -0.000188 |  0.8602   |  0.07926  |  13.87    |  10.77    |  0.9425   |
|  8        | -0.000188 |  0.987    |  0.2585   |  11.31    |  10.94    |  0.9903   |
|  9        | -0.000188 |  0.4093   |  0.01816  |  14.2     |  5.593    |  0.7637   |
|  10       | -0.000188 |  0.6155   |  0.02749  |  3.1

|  58       | -0.000188 |  0.4512   |  0.2074   |  14.96    |  19.74    |  0.8411   |
|  59       | -0.000188 |  0.7699   |  0.2874   |  3.031    |  19.95    |  0.5701   |
|  60       | -0.000188 |  0.9297   |  0.006135 |  14.65    |  3.108    |  0.9436   |
|  61       | -0.000188 |  0.6947   |  0.06474  |  3.058    |  3.074    |  0.7434   |
|  62       | -0.000188 |  0.5922   |  0.2985   |  3.104    |  3.262    |  0.9843   |
|  63       | -0.000188 |  0.6263   |  0.2512   |  14.93    |  19.72    |  0.8934   |
|  64       | -0.000188 |  0.4398   |  0.1639   |  14.86    |  3.418    |  0.8121   |
|  65       | -0.000188 |  0.7236   |  0.08866  |  3.557    |  19.97    |  0.7452   |
|  66       | -0.000188 |  0.8983   |  0.2268   |  14.48    |  3.015    |  0.8736   |
|  67       | -0.000188 |  0.9839   |  0.1998   |  14.92    |  19.68    |  0.8236   |
|  68       | -0.000188 |  0.9274   |  0.296    |  3.548    |  19.56    |  0.618    |
|  69       | -0.000188 |  0.4861   |  0.007605 |  4.1